In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163485 sha256=7d7adaa3346d111626f8f31c931c6511c9ee5f1f33c8f5dfea156522b587d4e9
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load mobile dataset
mobiles = pd.read_csv('mobile.csv')  # Replace with the actual filename

# Display the first few rows of the dataset
mobiles.head()

# Check the shape of the mobiles dataset
print(mobiles.shape)

# Check for missing values in the mobiles dataset
mobiles.isnull().sum()

# Check for duplicated rows in the mobiles dataset
mobiles.duplicated().sum()

## Popularity Based Recommender System
num_rating_df = mobiles.groupby('model').count()['rating'].reset_index()
num_rating_df.rename(columns={'rating': 'num_ratings'}, inplace=True)

avg_rating_df = mobiles.groupby('model').mean()['rating'].reset_index()
avg_rating_df.rename(columns={'rating': 'avg_rating'}, inplace=True)

popular_df = num_rating_df.merge(avg_rating_df, on='model')
popular_df = popular_df[popular_df['num_ratings'] >= 50].sort_values('avg_rating', ascending=False).head(50)

# Include additional columns from the original mobiles DataFrame
popular_df = popular_df.merge(mobiles[['model', 'price', 'rating', 'ram', 'camera']], on='model').drop_duplicates('model')[['model', 'price', 'rating', 'ram', 'camera', 'num_ratings', 'avg_rating']]

# Save the popular mobiles dataframe
pickle.dump(popular_df, open('popular_mobiles.pkl', 'wb'))

## Collaborative Filtering Based Recommender System


# Create the user-item matrix
pt = mobiles.pivot_table(index='model', columns='index', values='rating', fill_value=0)

# Calculate similarity scores using cosine similarity
similarity_scores = cosine_similarity(pt)

# Define a function to recommend similar mobiles
def recommend_mobile(model_name):
    index = np.where(pt.index == model_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = mobiles[mobiles['model'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('model')[['model', 'price', 'rating', 'ram', 'camera']].values[0]))

        data.append(item)

    return data

# Save collaborative filtering data
pickle.dump(pt, open('pt_mobiles.pkl', 'wb'))
pickle.dump(similarity_scores, open('similarity_scores_mobiles.pkl', 'wb'))




# Define a function to recommend similar mobiles
def recommend_mobile(model_name):
    index = np.where(pt.index == model_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = mobiles[mobiles['model'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('model')[['model', 'price', 'rating', 'ram', 'camera']].values[0]))

        data.append(item)

    return data

# Save collaborative filtering data
pickle.dump(pt, open('pt_mobiles.pkl', 'wb'))

# Assuming you have already loaded the collaborative filtering data
pt = pickle.load(open('pt_mobiles.pkl', 'rb'))
similarity_scores = pickle.load(open('similarity_scores_mobiles.pkl', 'rb'))

# Example: Recommend mobiles similar to a specific mobile
example_mobile = 'Samsung Galaxy S20'

# Call the recommend_mobile function
recommendations = recommend_mobile(example_mobile)

# Display the recommendations
print(f"Recommended mobiles similar to '{example_mobile}':")
for mobile in recommendations:
    print(f"Model: {mobile[0]}, Price: {mobile[1]}, Rating: {mobile[2]}, RAM: {mobile[3]}, Camera: {mobile[4]}")




(1020, 6)


<ipython-input-10-124c40d8e89b>:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_rating_df = mobiles.groupby('model').mean()['rating'].reset_index()


Recommended mobiles similar to 'Samsung Galaxy S20':
Model: Apple iPhone 11, Price: ₹38,999, Rating: 73.0, RAM: 4 GB RAM, 64 GB inbuilt, Camera: 12 MP + 12 MP Dual Rear & 12 MP Front Camera
Model: Apple iPhone 11 (128GB), Price: ₹46,999, Rating: 75.0, RAM: 4 GB RAM, 128 GB inbuilt, Camera: 12 MP + 12 MP Dual Rear & 12 MP Front Camera
Model: Apple iPhone 11 Pro Max, Price: ₹1,09,900, Rating: 77.0, RAM: 4 GB RAM, 64 GB inbuilt, Camera: 12 MP + 12 MP + 12 MP Triple Rear & 12 MP Front Camera
Model: Apple iPhone 12, Price: ₹51,999, Rating: 74.0, RAM: 4 GB RAM, 64 GB inbuilt, Camera: Memory Card Not Supported
